In [3]:
from os import listdir
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV,StratifiedKFold, KFold, RandomizedSearchCV, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

plt.style.use('ggplot') #ggplot스타일 사용
plt.rc('axes', unicode_minus=False)

In [4]:
data_file_path = './open data'
model_path = './modeling_folder/'

In [362]:
pre_sub = pd.read_csv(os.path.join(data_file_path,'dacon_submission6.csv'))

In [25]:
tr = pd.read_csv(os.path.join(data_file_path,'train.csv'))
te = pd.read_csv(os.path.join(data_file_path,'test_x.csv'))
base_column = tr.drop(columns = 'index').columns.tolist()
submission = te[['index']]

In [28]:
pd.get_dummies(tr[['age_group', 'gender', 'race', 'religion']])

,age_group_+70s,age_group_10s,age_group_20s,age_group_30s,age_group_40s,age_group_50s,age_group_60s,gender_Female,gender_Male,race_Arab,...,religion_Buddhist,religion_Christian_Catholic,religion_Christian_Mormon,religion_Christian_Other,religion_Christian_Protestant,religion_Hindu,religion_Jewish,religion_Muslim,religion_Other,religion_Sikh
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
45528,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
45529,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
45530,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
tr.select_dtypes('object').columns.tolist()

['age_group', 'gender', 'race', 'religion']

In [915]:
tr.columns

Index(['index', 'QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA',
       'QeE', 'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA',
       'QjE', 'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA',
       'QoE', 'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA',
       'QtE', 'age_group', 'education', 'engnat', 'familysize', 'gender',
       'hand', 'married', 'race', 'religion', 'tp01', 'tp02', 'tp03', 'tp04',
       'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban', 'voted',
       'wf_01', 'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05',
       'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12', 'wr_13'],
      dtype='object')

In [940]:
# 원핫인코딩을 진행하면 성능이 어떻게 바뀌지?
# mean encoding을 시행하면?
ob_col_ls = tr.select_dtypes('object').columns.tolist()

le = LabelEncoder()
for col in ob_col_ls:
    le.fit(tr[col])
    tr[col] = le.transform(tr[col])
    te[col] = le.transform(te[col])

# Feature engineering

In [8]:
def col_extract(df):
    col_ls = []
    for col in df.columns.tolist():
        if col[0] == 'Q' and col[-1] == 'E':
            col_ls.append(col)
            
    return col_ls

def distribution_plot(df,col):
    plt.figure(figsize = (15,10))
    sns.distplot(df[df['voted'] == 2][col], hist = False, kde = True,
                         kde_kws = {'shade': True, 'linewidth': 1}, label = 'non_voted')
    sns.distplot(df[df['voted'] == 1][col], hist = False, kde = True,
                     kde_kws = {'shade': True, 'linewidth': 1}, label = 'voted')
#     plt.hist(df[df['voted'] == 2][col], label = 'non_voted')
#     plt.hist(df[df['voted'] == 1][col], label = 'voted')
    plt.show()
    
def col_extract2(df,string):
    col_ls = []
    for col in df.columns.tolist():
        if string in col:
            col_ls.append(col)
            
    return col_ls

def sequence_column_extract(df,string):
    col_ls = df.columns.tolist()
    col_ls2 = []
    for col in col_ls:
        if string in col:
            col_ls2.append(col)
    
    return col_ls2

def correlation_extract(df, threshold):
    cor_df = df.corr().drop(columns = 'index')
    col_list = cor_df.columns
    
    for idx, col in enumerate(col_list):
        idx_ls = cor_df[abs(cor_df[col])>=threshold].index.tolist()
        print(f'{col} : {idx_ls}')
        
def col_extract_QA(df):
    col_ls = []
    for col in df.columns.tolist():
        if col[0] == 'Q' and col[-1] == 'A':
            col_ls.append(col)
            
    return col_ls

## 다중 공선성 변수 체크
    - 임계점: 0.8

In [12]:
correlation_extract(tr,0.3)

QaA : ['QaA', 'QcA', 'QdA', 'QkA', 'QnA', 'QqA']
QaE : ['QaE']
QbA : ['QbA', 'QcA', 'QhA', 'QiA', 'QjA', 'QkA', 'QmA', 'QnA', 'QoA', 'QpA', 'QqA', 'QsA', 'QtA']
QbE : ['QbE']
QcA : ['QaA', 'QbA', 'QcA', 'QiA', 'QjA', 'QkA', 'QmA', 'QnA', 'QoA', 'QpA', 'QqA', 'QtA']
QcE : ['QcE']
QdA : ['QaA', 'QdA', 'QnA', 'QqA']
QdE : ['QdE']
QeA : ['QeA']
QeE : ['QeE']
QfA : ['QfA', 'QgA', 'QiA', 'QkA', 'QmA', 'QnA', 'QoA', 'QrA', 'QsA']
QfE : ['QfE']
QgA : ['QfA', 'QgA', 'QkA', 'QnA', 'QqA']
QgE : ['QgE']
QhA : ['QbA', 'QhA', 'QjA', 'QkA', 'QpA', 'QtA']
QhE : ['QhE']
QiA : ['QbA', 'QcA', 'QfA', 'QiA', 'QjA', 'QkA', 'QmA', 'QnA', 'QoA', 'QqA', 'QrA', 'QsA', 'QtA', 'tp07']
QiE : ['QiE']
QjA : ['QbA', 'QcA', 'QhA', 'QiA', 'QjA', 'QkA', 'QmA', 'QoA', 'QpA', 'QqA', 'QtA']
QjE : ['QjE']
QkA : ['QaA', 'QbA', 'QcA', 'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 'QkA', 'QmA', 'QnA', 'QoA', 'QpA', 'QqA', 'QrA', 'QsA', 'QtA', 'tp07']
QkE : ['QkE']
QlA : ['QlA']
QlE : ['QlE']
QmA : ['QbA', 'QcA', 'QfA', 'QiA', 'QjA', 'QkA

## 성능개선 변수

In [13]:
# 각 로우별 응답 시간 통계
# 응답시간 합
tr['Q_E_sum'] = tr[col_extract(tr)].sum(axis=1)
# 응답시간 평균
#tr['Q_E_mean'] = tr[col_extract(tr)].mean(axis=1)
# 응답시간 중앙값
tr['Q_E_median'] = tr[col_extract(tr)].median(axis=1)

# 성능개선 확인해봐야 함
# 편차 데이터 생성
# max - min diff -> 확인해봐야함
# tr['Q_E_diff'] = tr[col_extract(tr)].max(axis=1) - tr[col_extract(tr)].min(axis=1) 


# 의미가 있을지 모르겠는데 부여한 점수 대비 소요된 시간
# for col in col_extract(tr):
#     t
# tr['temp'] = tr['QaA'] / tr['QaE']
# sns.kdeplot(tr[tr['voted'] == 1]['temp'], label = 'vote')
# sns.kdeplot(tr[tr['voted'] == 2]['temp'], label = 'non_vote')

# 응답시간분포는 거의비슷

te['Q_E_sum'] = te[col_extract(te)].sum(axis=1)
#te['Q_E_mean'] = te[col_extract(te)].mean(axis=1)
te['Q_E_median'] = te[col_extract(te)].median(axis=1)

# te['Q_E_diff'] = te[col_extract(tr)].max(axis=1) - te[col_extract(tr)].min(axis=1) 


In [14]:
# 교육수준 less than high school 변수
# 성능향상에 기여
tr['education_ft1'] = tr['education'].apply(lambda x: 2 if x == 1 else 1)
# 교육수준 1,2 묶고 / 3,4 묶고
tr['education_ft2'] = tr['education'].apply(lambda x: 2 if x in [1,2] else 1)

te['education_ft1'] = te['education'].apply(lambda x: 2 if x == 1 else 1)
te['education_ft2'] = te['education'].apply(lambda x: 2 if x in [1,2] else 1)

In [15]:
# married투표를 하지 않은 비율
# 성능향상에 기여
temp = tr.groupby(['married','voted'])['index'].count().reset_index()
temp2_married = temp[temp['voted'] == 2].reset_index()
married_value = temp[temp['voted'] == 2]['index'].reset_index()['index'] / temp.groupby('married')['index'].sum().reset_index()['index']
temp2_married['married_value'] = married_value

married_ratio_encoding = temp2_married[['married','married_value']]
tr = pd.merge(tr, married_ratio_encoding, on = 'married', how = 'left')
te = pd.merge(te, married_ratio_encoding, on = 'married', how = 'left')

In [16]:
# 연령대별 투표하지 않는 비율
# 성능향상에 기여
temp = tr.groupby(['age_group','voted'])['index'].count().reset_index()
temp2_age_group = temp[temp['voted'] == 2].reset_index()
age_group_value = temp[temp['voted'] == 2]['index'].reset_index()['index'] / temp.groupby('age_group')['index'].sum().reset_index()['index']
temp2_age_group['age_group_value'] = age_group_value

age_group_ratio_encoding = temp2_age_group[['age_group','age_group_value']]
tr = pd.merge(tr, age_group_ratio_encoding, on = 'age_group', how = 'left')
te = pd.merge(te, age_group_ratio_encoding, on = 'age_group', how = 'left')

In [17]:
tr['age_married_value_sum'] = tr['age_group_value'] + tr['married_value']
tr['age_married_value_divide'] = tr['age_group_value'] / tr['married_value']

te['age_married_value_sum'] = te['age_group_value'] + te['married_value']
te['age_married_value_divide'] = te['age_group_value'] / te['married_value']

In [18]:
# 성능향상에 기여
# 해당 단어의 정의를 하는 합산
tr['wr_sum'] = tr[col_extract2(tr,'wr')].sum(axis=1)
# 허구인 단어의 정의를 합산
tr['wf_sum'] = tr[col_extract2(tr,'wf')].sum(axis=1)
# 해당 단어 정의 + 허구 단어 정의 
# 성능향상에 기여
tr['wr_wr_sum'] = tr['wr_sum'] + tr['wr_sum']
tr['wf_wf_sum'] = tr['wf_sum'] + tr['wf_sum']
tr['wr_wf_sum'] = tr['wr_sum'] + tr['wf_sum']

# diff변수도 추가 해보기
tr['wr_wf_diff'] = tr['wr_sum'] - tr['wf_sum']
tr['wr_wf_mul'] = tr['wr_sum'] * tr['wf_sum']

te['wr_sum'] = te[col_extract2(te,'wr')].sum(axis=1)
te['wf_sum'] = te[col_extract2(te,'wf')].sum(axis=1)

te['wr_wr_sum'] = te['wr_sum'] + te['wr_sum']
te['wf_wf_sum'] = te['wf_sum'] + te['wf_sum']
te['wr_wf_sum'] = te['wr_sum'] + te['wf_sum']

te['wr_wf_diff'] = te['wr_sum'] - te['wf_sum']
te['wr_wf_mul'] = te['wr_sum'] * te['wf_sum']


In [19]:
# 마키아벨리니즘 테스트 스코어 컬럼 추가
# Q_A 관련 변수들의 값이 모두 변하게 되었는데 이대로 돌려보자
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

correlations = tr[Answers].corr(method = 'spearman')
# sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    tr[flip] = 6 - tr[flip]
    te[flip] = 6 - te[flip]
    
correlations = tr[Answers].corr(method = 'spearman')
# sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    tr[flip] = 6 - tr[flip]
    te[flip] = 6 - te[flip]
correlations = tr[Answers].corr(method='spearman')
# sns.heatmap(correlations, cmap="coolwarm", square=True, center=0)

tr['Mach_score'] = tr[Answers].mean(axis = 1)
te['Mach_score'] = te[Answers].mean(axis = 1)

In [21]:
# 시간 변수들 로그화
# sns.kdeplot(np.log10(tr['QfE']))
qe_col = col_extract(tr)

for col in qe_col:
    tr[col + '_log'] = np.log10(tr[col])
    
for col in qe_col:
    te[col + '_log'] = np.log10(te[col])

tr = tr.drop(columns = qe_col)    
te = te.drop(columns = qe_col)

columns_ls = [
    'QaE', 'QbE', 'QcE', 'QdE', 'QeE',
     'QfE', 'QgE', 'QhE', 'QiE', 'QjE',
     'QkE', 'QlE', 'QmE', 'QnE', 'QoE',
     'QpE', 'QqE', 'QrE', 'QsE', 'QtE',
     'hand'
]
tr = tr.drop(columns = columns_ls)
te = te.drop(columns = columns_ls)

94

## 성능 비개선 변수

In [ ]:
# 전혀 쓸모없
tr['Q_E_std'] = tr[col_extract(tr)].std(axis = 1)

# max, min값 생성
# min만 넣어서
tr['Q_E_min'] = tr[col_extract(tr)].min(axis = 1)
# 전혀 쓸모없
tr['Q_E_max'] = tr[col_extract(tr)].max(axis = 1)

# tr['diff'] 도 성능이 굉장히 떨어질 것이라 예상

In [333]:
# edu 투표를 하지 않을 비율
# 성능향상에 비기여
temp = tr.groupby(['education','voted'])['index'].count().reset_index()
temp2_edu = temp[temp['voted'] == 2].reset_index()
edu_value = temp[temp['voted'] == 2]['index'].reset_index()['index'] / temp.groupby('education')['index'].sum().reset_index()['index']
temp2_edu['edu_value'] = edu_value

edu_ratio_encoding = temp2_edu[['education','edu_value']]
tr = pd.merge(tr, edu_ratio_encoding, on = 'education', how = 'left')
te = pd.merge(te, edu_ratio_encoding, on = 'education', how = 'left')

In [267]:
# 결혼여부 1,2 묶고 / 3,4 묶고
# 성능향상에 비기여
tr['married_ft1'] = tr['married'].apply(lambda x: 2 if x == 1 else 1)

te['married_ft1'] = te['married'].apply(lambda x: 2 if x == 1 else 1)

In [372]:
# 연령대구분
# 10대, 20대-30대, 40대 이상
# 성능향상에 비기여
tr['age_group_ft1'] = tr['age_group'].apply(lambda x: 1 if x in [1] else (2 if x in [2,3] else 3))

te['age_group_ft1'] = te['age_group'].apply(lambda x: 1 if x in [1] else (2 if x in [2,3] else 3))

In [ ]:
# 카테고리컬한 변수를 활용하여 어떻게 피쳐를 생성할 수 있을지 고민


# 성능개선 여부 확인해봐야 함
# 카테고리 변수들 combination
# 인종 + 종교
temp_df = tr.groupby(['race','religion','voted'])['index'].count().reset_index()

temp_df['rac_rel_com'] = temp_df['race'].astype('str') + temp_df['religion'].astype('str')

temp_df2 = temp_df.groupby('rac_rel_com')\
    ['index'].sum().\
    reset_index(name = 'idx_sum')

temp_df3 = pd.merge(
    temp_df2, 
    temp_df[temp_df['voted'] == 2],
    on = 'rac_rel_com',
    how = 'left'
)

temp_df3 = temp_df3.fillna(0)

temp_df3['ratio'] = temp_df3['index'] / temp_df3['idx_sum']
#temp_df3['rac_rel_index'] = temp_df3['ratio'].apply(lambda x: 1 if x < 0.4 else (2 if x > 0.6 else 0))
#temp_df4 = temp_df3[(temp_df3['idx_sum'] >= 50) & (temp_df3['ratio'] >= 0.6)]
temp_df3['rac_rel_index'] = temp_df3['ratio'].apply(lambda x: 2 if x > 0.6 else 1)

tr['rac_rel_com'] = tr['race'].astype('str') + tr['religion'].astype('str')
te['rac_rel_com'] = te['race'].astype('str') + te['religion'].astype('str')

tr = pd.merge(
    tr,
    temp_df3[['rac_rel_com','rac_rel_index']],
    on = 'rac_rel_com',
    how = 'left'
)

te = pd.merge(
    te,
    temp_df3[['rac_rel_com','rac_rel_index']],
    on = 'rac_rel_com',
    how = 'left'
)

te.drop(columns = ['rac_rel_com'], inplace = True)
tr.drop(columns = ['rac_rel_com'], inplace = True)


# 연령대 + 결혼여부
temp_df = tr.groupby(['age_group','married','voted'])['index'].count().reset_index()

temp_df['age_mar_com'] = temp_df['age_group'].astype('str') + temp_df['married'].astype('str')

temp_df2 = temp_df.groupby('age_mar_com')\
    ['index'].sum().\
    reset_index(name = 'idx_sum')

temp_df3 = pd.merge(
    temp_df2, 
    temp_df[temp_df['voted'] == 2],
    on = 'age_mar_com',
    how = 'left'
)

temp_df3 = temp_df3.fillna(0)

temp_df3['ratio'] = temp_df3['index'] / temp_df3['idx_sum']
#temp_df3['age_mar_index'] = temp_df3['ratio'].apply(lambda x: 1 if x < 0.4 else (2 if x > 0.6 else 0))
temp_df3['age_mar_index'] = temp_df3['ratio'].apply(lambda x: 2 if x > 0.6 else 1)
#temp_df4 = temp_df3[(temp_df3['idx_sum'] >= 50) & (temp_df3['ratio'] >= 0.5)]

tr['age_mar_com'] = tr['age_group'].astype('str') + tr['married'].astype('str')
te['age_mar_com'] = te['age_group'].astype('str') + te['married'].astype('str')

tr = pd.merge(
    tr,
    temp_df3[['age_mar_com','age_mar_index']],
    on = 'age_mar_com',
    how = 'left'
)

te = pd.merge(
    te,
    temp_df3[['age_mar_com','age_mar_index']],
    on = 'age_mar_com',
    how = 'left'
)

te.drop(columns = ['age_mar_com'], inplace = True)
tr.drop(columns = ['age_mar_com'], inplace = True)

te['age_mar_index'] = te['age_mar_index'].fillna(0)
te['age_mar_index'] = te['age_mar_index'].astype('int64')

In [516]:
# tp변수도 기존 tp변수 제외하고 mean encoding한 것으로 제출해보기
# tp 데이터 관련 FE
# 6,9,3,7,8,1,10
# 현상태에서 성능향상에 비기여

#tr.groupby(['tp08','voted'])['index'].count().reset_index()

# tp관련해서 점수를 만들어도 될 듯(sum)
seq_col_ls = sequence_column_extract(tr,'tp')
tr['tp_sum'] = tr[seq_col_ls].sum(axis = 1)
te['tp_sum'] = te[seq_col_ls].sum(axis = 1)

# 투표를 하지 않을 확률
# seq_col_ls2 = ['tp01',
#              'tp03',
#              'tp06',
#              'tp07',
#              'tp08',
#              'tp09',
#              'tp10']

# for col in seq_col_ls2:
#     temp = tr.groupby([col,'voted'])['index'].count().reset_index()
#     temp2 = temp[temp['voted'] == 2].reset_index()
#     temp_value = temp[temp['voted'] == 2]['index'].reset_index()['index'] / temp.groupby(col)['index'].sum().reset_index()['index']
#     temp2[col+'non_voted_value'] = temp_value

#     temp_encoding = temp2[[col,col+'non_voted_value']]
#     tr = pd.merge(tr, temp_encoding, on = col, how = 'left')
#     te = pd.merge(te, temp_encoding, on = col, how = 'left')

In [ ]:
def QA_voted_dist(df, qa_col, numbering):
    temp = tr.groupby([qa_col[numbering], 'voted'])['index'].count().reset_index()
    print(temp)
    
for idx in range(len(qa_col_list)):
    print(qa_col_list[idx])
    QA_voted_dist(tr, qa_col_list, idx)

In [ ]:
# Qa 답변별 투표여부 분포
qa_col_list = col_extract_QA(tr)

# voted = 2 mean encoding
# Q_A 관련해서 mean_encoding는 성능이 더 떨어짐
for col in qa_col_list:
    temp = tr.groupby([col,'voted'])['index'].count().reset_index()
    temp2 = temp[temp['voted'] == 2].reset_index()
    temp_value = temp[temp['voted'] == 2]['index'].reset_index()['index'] / temp.groupby(col)['index'].sum().reset_index()['index']
    temp2[col+'non_voted_value'] = temp_value

    temp_encoding = temp2[[col,col+'non_voted_value']]
    tr = pd.merge(tr, temp_encoding, on = col, how = 'left')
    te = pd.merge(te, temp_encoding, on = col, how = 'left')

In [ ]:
# tp변수들로 하여금 점수를 부여한것의 분포도 그리기

# tp변수들 기준으로 답장별 투표, 비투표 분포차이가 비슷한 것들만 뽑아서 통계 데이터
# 생성
seq_col_ls = sequence_column_extract(tr,'tp')

for col in seq_col_ls:
    for i in [1,2]:
        sns.kdeplot(
            tr[tr['voted'] == i][col],
            label=i,
            legend=True
        )
    plt.title(col)
    plt.show()

In [ ]:
tr['tp02_category'] = tr['tp02'].apply(
    lambda x: 2 if x in [0,1,2,3] else (1 if x in [5,6] else 0)
)

tr['tp03_category'] = tr['tp03'].apply(
    lambda x: 2 if x in [2,3,4,5,6] else (1 if x in [0] else 0)
)

tr['tp04_category'] = tr['tp04'].apply(
    lambda x: 2 if x in [0,1] else (1 if x in [4,5,6] else 0)
)

tr['tp06_category'] = tr['tp06'].apply(
    lambda x: 2 if x in [0,1,2] else (1 if x in [5,6] else 0)
)

tr['tp07_category'] = tr['tp07'].apply(
    lambda x: 2 if x in [2,3,4,5,6] else (1 if x in [0,1] else 0)
)

tr['tp08_category'] = tr['tp08'].apply(
    lambda x: 2 if x in [0,1,2,3] else (1 if x in [5,6] else 0)
)

tr['tp09_category'] = tr['tp09'].apply(
    lambda x: 2 if x in [5,6] else (1 if x in [0,1] else 0)
)

# tr['tp04'] + tr['tp06'] + tr['tp08']


te['tp02_category'] = te['tp02'].apply(
    lambda x: 2 if x in [0,1,2,3] else (1 if x in [5,6] else 0)
)

te['tp03_category'] = te['tp03'].apply(
    lambda x: 2 if x in [2,3,4,5,6] else (1 if x in [0] else 0)
)

te['tp04_category'] = te['tp04'].apply(
    lambda x: 2 if x in [0,1] else (1 if x in [4,5,6] else 0)
)

te['tp06_category'] = te['tp06'].apply(
    lambda x: 2 if x in [0,1,2] else (1 if x in [5,6] else 0)
)

te['tp07_category'] = te['tp07'].apply(
    lambda x: 2 if x in [2,3,4,5,6] else (1 if x in [0,1] else 0)
)

te['tp08_category'] = te['tp08'].apply(
    lambda x: 2 if x in [0,1,2,3] else (1 if x in [5,6] else 0)
)

te['tp09_category'] = te['tp09'].apply(
    lambda x: 2 if x in [5,6] else (1 if x in [0,1] else 0)
)

In [ ]:
# kmeans 변수: float 변수만 넣는 것이 좋나?
# qa / qe는 군집을 제대로 나누지 못함

tp_col = sequence_column_extract(tr,'tp')

b = tr[tp_col]
b['training'] = 'train'
c = te[tp_col]
c['training'] = 'test'

d = pd.concat([b,c],axis = 0)
e = d.drop(columns = 'training')

numeric_var = e.columns.tolist()

ks = range(1,10)

inertias = []
for k in ks:

    model = KMeans(n_clusters=k)

    model.fit(e)

    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

model = KMeans(n_clusters=3,algorithm='auto')
model.fit(e)
predict = pd.DataFrame(model.predict(e))

d['tp_cluster'] = predict[0]

tr['tp_cluster'] = d[d['training'] == 'train']['tp_cluster']
te['tp_cluster'] = d[d['training'] == 'test']['tp_cluster']

In [ ]:
# kmeans 변수: float 변수만 넣는 것이 좋나?
# qa / qe는 군집을 제대로 나누지 못함

wr_col = sequence_column_extract(tr,'wr')

b = tr[wr_col]
b['training'] = 'train'
c = te[wr_col]
c['training'] = 'test'

d = pd.concat([b,c],axis = 0)
e = d.drop(columns = 'training')

numeric_var = e.columns.tolist()

ks = range(1,10)

inertias = []
for k in ks:

    model = KMeans(n_clusters=k)

    model.fit(e)

    inertias.append(model.inertia_)
    
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

model = KMeans(n_clusters=5,algorithm='auto')
model.fit(e)
predict = pd.DataFrame(model.predict(e))

d['wr_cluster'] = predict[0]

tr['wr_cluster'] = d[d['training'] == 'train']['wr_cluster']
te['wr_cluster'] = d[d['training'] == 'test']['wr_cluster']

In [ ]:
# 데이터 더미변수화
tr2 = pd.get_dummies([dummy_col_ls])

In [ ]:
# 데이터 더미화
tr2 = pd.read_csv(os.path.join(data_file_path,'train.csv'))
te2 = pd.read_csv(os.path.join(data_file_path,'test_x.csv'))

tr = pd.concat([tr,pd.get_dummies(tr2[['age_group', 'gender', 'race', 'religion']])],axis = 1)
tr = tr.drop(columns = ['age_group', 'gender', 'race', 'religion'])

te = pd.concat([te,pd.get_dummies(te2[['age_group', 'gender', 'race', 'religion']])],axis = 1)
te = te.drop(columns = ['age_group', 'gender', 'race', 'religion'])


# 카테고리컬한 변수 전혀 필요 없었음
for col in ['age_group', 'education', 'engnat', 'familysize', 'gender', 'hand','married', 'race', 'religion','urban']:
    tr[col] = tr[col].astype('category')

for col in ['age_group', 'education', 'engnat', 'familysize', 'gender', 'hand','married', 'race', 'religion','urban']:
    te[col] = te[col].astype('category')

# 혹시모르는 변수 소거

In [563]:
# 변수 소거 임계치 0.7이상인 것들도 제거해보자
# wr_wf_sum만 제거해서 제출해보자
tr2 = tr.drop(columns = qa_col_list)
# tr2 = tr.drop(columns = 'age_group_value')

te2 = te.drop(columns = qa_col_list)
# te2 = te.drop(columns = 'age_group_value')

# 본격적인 변수 생성하는 부분
- 여기서 변수를 생성하고 두가지 부분으로 나눌 예정
    - 성능 개선
    - 성능 비개선

In [948]:
seq_col_ls = sequence_column_extract(tr,'tp')
tr['tp_mean'] = tr[seq_col_ls].mean(axis = 1)
te['tp_mean'] = te[seq_col_ls].mean(axis = 1)

tr['tp_median'] = tr[seq_col_ls].median(axis = 1)
te['tp_median'] = te[seq_col_ls].median(axis = 1)

In [ ]:
# 가설: 적극적인 사람이 투표를 잘할 것이다?
# 질문을 통해서 사람들의 적극성을 물어보는 척도로 사용되는 질문들을 추려 FE

# Data extract

In [23]:
# ver1: 현재까지 성능이 가장 좋았던 버전
# ver2: Q_A 변수 mean encoding 
# ver3: 실수로 그냥 보냄
# ver4: mean encoding 변수만 남기고 기존 Q_A변수 소거
def data_extract(tr, te):
    
    modeling_file_ls = []
    for file in os.listdir(model_path):
        if 'modeling' in file:
            modeling_file_ls.append(file)

    ver_num_ls = []
    for file in modeling_file_ls:
        ver_num_ls.append(re.sub('[^0-9]','',file))

    ver_num = int(pd.Series(list(set(ver_num_ls))).sort_values(ascending=False).iloc[0])
#     ver_num = 10
    tr.to_csv(
        os.path.join(model_path,'modeling_tr_ver{}.csv'.format(ver_num + 1)),
        index = False
    )
    te.to_csv(
        os.path.join(model_path,'modeling_te_ver{}.csv'.format(ver_num + 1)),
        index = False
    )

In [24]:
data_extract(tr,te)